In [1]:
import requests
import pandas as pd
import re
from html import unescape

In [2]:
dfs_sheets = {}
descriptions = []

In [3]:
def get_content_president(year, month, day):
  response = requests.get(f"https://www.bcb.gov.br/api/servico/sitebcb/agendadiretoria?lista=Agenda%20da%20Diretoria&inicioAgenda=%272023-02-27%27&fimAgenda=%27{year}-{month}-{day}%27")
  if (response.status_code == 200):
    data = response.json()
    if (data["conteudo"][0]["idAutoridade"] == 39):
      return data["conteudo"][0]
  return None

In [4]:
year = 2023
for month in range(2, 13):
  if month == 2:
    content = get_content_president(year, month, 28)
    description = content["descricao"].encode("utf-8").decode("unicode_escape")
    description_regex = re.sub(r'<[^>]+>', '', description)
    description_normalized = unescape(description_regex)
    descriptions.append(description_normalized)
    df = pd.DataFrame({
      "descricoes": descriptions.copy(),
    }, index=[28])
    descriptions.clear()
    dfs_sheets[f"{month}-{year}"] = df
  elif month in [3, 5, 7, 8, 10, 12]:
    for day in range(1, 32):
      content = get_content_president(year, month, day)
      description = content["descricao"].encode("utf-8").decode("unicode_escape")
      description_regex = re.sub(r'<[^>]+>', '', description)
      description_normalized = unescape(description_regex)
      descriptions.append(description_normalized)
    df = pd.DataFrame({
      "descricoes": descriptions.copy()
    }, index=range(1, len(descriptions) + 1))
    descriptions.clear()
    dfs_sheets[f"{month}-{year}"] = df
  elif month in [4, 6, 9, 11]:
    for day in range(1, 31):
      content = get_content_president(year, month, day)
      description = content["descricao"].encode("utf-8").decode("unicode_escape")
      description_regex = re.sub(r'<[^>]+>', '', description)
      description_normalized = unescape(description_regex)
      descriptions.append(description_normalized)
    df = pd.DataFrame({
      "descricoes": descriptions.copy()
    }, index=range(1, len(descriptions) + 1))
    descriptions.clear()
    dfs_sheets[f"{month}-{year}"] = df

UnicodeEncodeError: 'ascii' codec can't encode character '\xe3' in position 76: ordinal not in range(128)

In [ ]:
year = 2024
for month in range(1, 7):
  if month in [1, 3, 5]:
    for day in range(1, 32):
      content = get_content_president(year, month, day)
      description = content["descricao"].encode("utf-8").decode("unicode_escape")
      description_regex = re.sub(r'<[^>]+>', '', description)
      description_normalized = unescape(description_regex)
      descriptions.append(description_normalized)
    df = pd.DataFrame({
      "descricoes": descriptions.copy()
    }, index=range(1, len(descriptions) + 1))
    descriptions.clear()
    dfs_sheets[f"{month}-{year}"] = df
  elif month == 4:
    for day in range(1, 31):
      content = get_content_president(year, month, day)
      description = content["descricao"].encode("utf-8").decode("unicode_escape")
      description_regex = re.sub(r'<[^>]+>', '', description)
      description_normalized = unescape(description_regex)
      descriptions.append(description_normalized)
    df = pd.DataFrame({
      "descricoes": descriptions.copy()
    }, index=range(1, len(descriptions) + 1))
    descriptions.clear()
    dfs_sheets[f"{month}-{year}"] = df
  elif month == 2:
    for day in range(1, 30):
      content = get_content_president(year, month, day)
      description = content["descricao"].encode("utf-8").decode("unicode_escape")
      description_regex = re.sub(r'<[^>]+>', '', description)
      description_normalized = unescape(description_regex)
      descriptions.append(description_normalized)
    df = pd.DataFrame({
      "descricoes": descriptions.copy()
    }, index=range(1, len(descriptions) + 1))
    descriptions.clear()
    dfs_sheets[f"{month}-{year}"] = df
  elif month == 6:
    for day in range(1, 29):
      content = get_content_president(year, month, day)
      description = content["descricao"].encode("utf-8").decode("unicode_escape")
      description_regex = re.sub(r'<[^>]+>', '', description)
      description_normalized = unescape(description_regex)
      descriptions.append(description_normalized)
    df = pd.DataFrame({
      "descricoes": descriptions.copy()
    }, index=range(1, len(descriptions) + 1))
    descriptions.clear()
    dfs_sheets[f"{month}-{year}"] = df

In [ ]:
df_xlsx = pd.read_excel("agenda.xlsx")
df_xlsx_no_empty_lines = df_xlsx.dropna(how="any")
df_xlsx_no_empty_columns = df_xlsx_no_empty_lines.dropna(axis=1, how="any")
df_xlsx_no_empty_columns.to_excel("agenda.xlsx", index=False)

with pd.ExcelWriter("agend.xlsx") as writer:
  for sheet_name, df in dfs_sheets.items():
    df.to_excel(writer, sheet_name=sheet_name)